# Importer les librairies, et fichiers .py auxiliaires

In [1]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble
from xgboost_classifier import train_xgboost, train_xgboost_with_tfidf
from svc_classifier import *
from sgd_classifier import *
from sklearn.model_selection import StratifiedKFold

from save_output import save_output

In [2]:
data_preprocess = DataPreprocess()
X_train, y_train, X_test = data_preprocess.train, data_preprocess.label_train, data_preprocess.test

# Exploration de données

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 7)$:
- α: Lissage du postérieur de Bayes

Prétraitement: aucun

In [3]:
bayes_classifier = BayesClassifier()
best_lps, best_f1 = bayes_classifier.hyperparameter_tuning(X_train, y_train)
print(best_lps, best_f1)
tuned_bayes_classifier = BayesClassifier()
tuned_bayes_classifier.fit(X_train, y_train, best_lps)
predictions = np.array([tuned_bayes_classifier.predict(x_i) for x_i in X_test])
save_output(predictions, "bayes", best_lps, "")

100%|██████████| 13/13 [00:33<00:00,  2.57s/it]


0.55 0.5955466401311209
predictions saved in output/20241112/135537_bayes_0.55_.csv
already same predictions saved in submissions? 0.73295_output_labels_bayes_classifier_part_4.csv
already same predictions saved in output?
	 output/20241111/204741_bayes_0.55_stopwords.csv
	 output/20241112/135425_bayes_0.55_.csv
Number of 0: 1594
Number of 1: 762
Ratio of 1: 0.3234295415959253
Number of differences with bayes classifier submission: 67
Ratio of 1 in bayes submission: 0.29499151103565363


# Jalon 2)
# Naive Bayes Complement
K-Fold validation croisée stratifiée ($k = 5)$:
- α: Lissage du postérieur de Bayes
 
Prétraitement: TFIDF

In [4]:
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_10_iter", "tfidf")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....cnb__alpha=0.5619101782710437;, score=0.286 total time=   1.8s
[CV 2/5] END .....cnb__alpha=0.5619101782710437;, score=0.277 total time=   1.8s
[CV 3/5] END .....cnb__alpha=0.5619101782710437;, score=0.288 total time=   1.8s
[CV 4/5] END .....cnb__alpha=0.5619101782710437;, score=0.289 total time=   1.8s
[CV 5/5] END .....cnb__alpha=0.5619101782710437;, score=0.270 total time=   1.8s
[CV 1/5] END .....cnb__alpha=1.4261714596148742;, score=0.030 total time=   1.8s
[CV 2/5] END .....cnb__alpha=1.4261714596148742;, score=0.004 total time=   1.8s
[CV 3/5] END .....cnb__alpha=1.4261714596148742;, score=0.013 total time=   1.8s
[CV 4/5] END .....cnb__alpha=1.4261714596148742;, score=0.017 total time=   1.8s
[CV 5/5] END .....cnb__alpha=1.4261714596148742;, score=0.026 total time=   1.8s
[CV 1/5] END .....cnb__alpha=1.0980909127171077;, score=0.071 total time=   1.8s
[CV 2/5] END .....cnb__alpha=1.0980909127171077;

Prétraitement: Sans prétraitement

In [5]:
complement_naive_bayes = train_cnb(X_train, y_train)
predictions = complement_naive_bayes.predict(X_test)
save_output(predictions, "cnb", "random_search_10_iter", "no_preprocessing")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....cnb__alpha=0.5619101782710437;, score=0.582 total time=   0.6s
[CV 2/5] END .....cnb__alpha=0.5619101782710437;, score=0.600 total time=   0.5s
[CV 3/5] END .....cnb__alpha=0.5619101782710437;, score=0.584 total time=   0.5s
[CV 4/5] END .....cnb__alpha=0.5619101782710437;, score=0.576 total time=   0.5s
[CV 5/5] END .....cnb__alpha=0.5619101782710437;, score=0.557 total time=   0.5s
[CV 1/5] END .....cnb__alpha=1.4261714596148742;, score=0.590 total time=   0.6s
[CV 2/5] END .....cnb__alpha=1.4261714596148742;, score=0.596 total time=   0.5s
[CV 3/5] END .....cnb__alpha=1.4261714596148742;, score=0.585 total time=   0.5s
[CV 4/5] END .....cnb__alpha=1.4261714596148742;, score=0.543 total time=   0.5s
[CV 5/5] END .....cnb__alpha=1.4261714596148742;, score=0.561 total time=   0.5s
[CV 1/5] END .....cnb__alpha=1.0980909127171077;, score=0.594 total time=   0.5s
[CV 2/5] END .....cnb__alpha=1.0980909127171077;

Prétraitement: TFIDF, réduction par arbre, stopwords retirés

In [6]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indices_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indices_features]
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, data_preprocess.label_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_10_iter", "tree_reduction_stopwords_tfidf")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....cnb__alpha=0.5619101782710437;, score=0.568 total time=   0.1s
[CV 2/5] END .....cnb__alpha=0.5619101782710437;, score=0.563 total time=   0.1s
[CV 3/5] END .....cnb__alpha=0.5619101782710437;, score=0.562 total time=   0.1s
[CV 4/5] END .....cnb__alpha=0.5619101782710437;, score=0.551 total time=   0.1s
[CV 5/5] END .....cnb__alpha=0.5619101782710437;, score=0.558 total time=   0.1s
[CV 1/5] END .....cnb__alpha=1.4261714596148742;, score=0.582 total time=   0.1s
[CV 2/5] END .....cnb__alpha=1.4261714596148742;, score=0.555 total time=   0.1s
[CV 3/5] END .....cnb__alpha=1.4261714596148742;, score=0.572 total time=   0.1s
[CV 4/5] END .....cnb__alpha=1.4261714596148742;, score=0.554 total time=   0.1s
[CV 5/5] END .....cnb__alpha=1.4261714596148742;, score=0.539 total time=   0.1s
[CV 1/5] END .....cnb__alpha=1.0980909127171077;, score=0.579 total time=   0.1s
[CV 2/5] END .....cnb__alpha=1.0980909127171077;

Prétraitement: TFIDF, réduction par arbre, stopwords retirés, suréchantillonnage SMOTE

In [7]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indices_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indices_features]
X_train, y_train = smote_oversampling(X_train, data_preprocess.label_train)

complement_naive_bayes = train_cnb(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)

save_output(predictions, "cnb", "random_search_10_iter", "tree_reduction_stopwords_tfidf_smote")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....cnb__alpha=0.5619101782710437;, score=0.634 total time=   0.0s
[CV 2/5] END .....cnb__alpha=0.5619101782710437;, score=0.651 total time=   0.0s
[CV 3/5] END .....cnb__alpha=0.5619101782710437;, score=0.632 total time=   0.0s
[CV 4/5] END .....cnb__alpha=0.5619101782710437;, score=0.658 total time=   0.0s
[CV 5/5] END .....cnb__alpha=0.5619101782710437;, score=0.700 total time=   0.0s
[CV 1/5] END .....cnb__alpha=1.4261714596148742;, score=0.634 total time=   0.0s
[CV 2/5] END .....cnb__alpha=1.4261714596148742;, score=0.649 total time=   0.0s
[CV 3/5] END .....cnb__alpha=1.4261714596148742;, score=0.632 total time=   0.0s
[CV 4/5] END .....cnb__alpha=1.4261714596148742;, score=0.654 total time=   0.0s
[CV 5/5] END .....cnb__alpha=1.4261714596148742;, score=0.698 total time=   0.0s
[CV 1/5] END .....cnb__alpha=1.0980909127171077;, score=0.634 total time=   0.0s
[CV 2/5] END .....cnb__alpha=1.0980909127171077;

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [9]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

complement_naive_bayes = train_cnb(X_train_undersampled, y_train_undersampled)
predictions = complement_naive_bayes.predict(data_preprocess.test)

save_output(predictions, "cnb", "random_search_10_iter", "stopwords_cum-sum_undersampled")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....cnb__alpha=0.5619101782710437;, score=0.704 total time=   0.1s
[CV 2/5] END .....cnb__alpha=0.5619101782710437;, score=0.712 total time=   0.1s
[CV 3/5] END .....cnb__alpha=0.5619101782710437;, score=0.687 total time=   0.1s
[CV 4/5] END .....cnb__alpha=0.5619101782710437;, score=0.701 total time=   0.1s
[CV 5/5] END .....cnb__alpha=0.5619101782710437;, score=0.696 total time=   0.1s
[CV 1/5] END .....cnb__alpha=1.4261714596148742;, score=0.709 total time=   0.1s
[CV 2/5] END .....cnb__alpha=1.4261714596148742;, score=0.716 total time=   0.1s
[CV 3/5] END .....cnb__alpha=1.4261714596148742;, score=0.691 total time=   0.1s
[CV 4/5] END .....cnb__alpha=1.4261714596148742;, score=0.711 total time=   0.1s
[CV 5/5] END .....cnb__alpha=1.4261714596148742;, score=0.695 total time=   0.1s
[CV 1/5] END .....cnb__alpha=1.0980909127171077;, score=0.703 total time=   0.1s
[CV 2/5] END .....cnb__alpha=1.0980909127171077;

# SVC
Grid Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- $\gamma$ : Coefficient du Noyau RBF
- $C$ : Terme de régularization pour la pénalité euclidienne.

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [11]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_params_, best_score_ = train_svc(X_train_undersampled, y_train_undersampled)
print(best_params_, best_score_)
svc = SVC(kernel='rbf', C=best_params_['C'], gamma=best_params_['gamma'])
svc.fit(X_train_undersampled, y_train_undersampled)
y_pred = svc.predict(data_preprocess.test)
params = f"C={best_params_['C']}, gamma={best_params_['gamma']}"
save_output(y_pred, "svm", params, "stopwords_cum-sum_undersampled")

{'gamma': array([0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01 , 0.011,
       0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019]), 'C': array([45, 46, 47, 48, 49, 50, 51, 52, 53, 54])}
Fitting 5 folds for each of 170 candidates, totalling 850 fits
[CV 1/5] END .................C=45, gamma=0.003;, score=0.635 total time=   0.3s
[CV 2/5] END .................C=45, gamma=0.003;, score=0.637 total time=   0.3s
[CV 3/5] END .................C=45, gamma=0.003;, score=0.636 total time=   0.3s
[CV 4/5] END .................C=45, gamma=0.003;, score=0.632 total time=   0.3s
[CV 5/5] END .................C=45, gamma=0.003;, score=0.632 total time=   0.3s
[CV 1/5] END .................C=45, gamma=0.004;, score=0.646 total time=   0.2s
[CV 2/5] END .................C=45, gamma=0.004;, score=0.634 total time=   0.2s
[CV 3/5] END .................C=45, gamma=0.004;, score=0.645 total time=   0.2s
[CV 4/5] END .................C=45, gamma=0.004;, score=0.641 total time=   0.2s
[CV 5/5] 

# SGDClassifier
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- loss: ModifiedHuber 
- penalty: ElaticNet 
- l1_ratio: Porportion de la perte d'ElasticNet qui est l1.
- $\alpha$: Poids attribué au terme de régularisation. Une plus grande valeur favorise que certains coefficients soient annulés (par l1) ou fortement adoucis (par l2) quand la pénalité est ElaticNet. 

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [4]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_sgd = train_sgd(X_train_undersampled, y_train_undersampled)
predictions_voter = best_sgd.predict(data_preprocess.test)
save_output(predictions_voter, "sgd", "random_search_10_iter", "stopwords_undersampling_cumulative_sum")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.524 total time=   2.2s
[CV 2/5] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.564 total time=   3.3s
[CV 3/5] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.525 total time=   2.0s
[CV 4/5] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.535 total time=   1.7s
[CV 5/5] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.544 total time=   3.2s
[CV 1/5] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.6s
[CV 2/5] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.6s
[CV 3/5] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.6s
[CV 4/5] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.6s
[CV 5/5] END model__

KeyboardInterrupt: 

# XGBoost
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- Learning rate: Taille du pas lors de la descente de critère sur la perte (gain de subdivision) 
- Nombre d'estimateurs (n_estimators): Nombre d'arbres de décision
- Profondeur maximale (max_depth): Profondeur maximale de chaque arbre (nombre de branchements max)
- sous-échantillon (subsample):  Proportion de données utilisées pour produire chaque arbre.

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [5]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

xgboost_classifier = train_xgboost(X_train_undersampled, y_train_undersampled)
predictions = xgboost_classifier.predict(data_preprocess.test)
save_output(predictions, "xgboost", "random_search_10_iter", "stopwords_undersampling_cumulative_sum")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END model__learning_rate=0.0849080237694725, model__max_depth=3, model__n_estimators=400, model__subsample=0.892797576724562;, score=0.644 total time=   3.5s
[CV 2/5] END model__learning_rate=0.0849080237694725, model__max_depth=3, model__n_estimators=400, model__subsample=0.892797576724562;, score=0.627 total time=   3.4s


KeyboardInterrupt: 

# Apprentissage par ensembles: CNB, XGBoost, Logistic Regression

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [3]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_10_iter", "stopwords_undersampling_cumulative_sum")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ...model__alpha=0.3851694527491273;, score=0.681 total time=   0.2s
[CV 2/5] END ...model__alpha=0.3851694527491273;, score=0.703 total time=   0.0s
[CV 3/5] END ...model__alpha=0.3851694527491273;, score=0.694 total time=   0.0s
[CV 4/5] END ...model__alpha=0.3851694527491273;, score=0.707 total time=   0.0s
[CV 5/5] END ...model__alpha=0.3851694527491273;, score=0.675 total time=   0.0s
[CV 1/5] END ...model__alpha=0.5016325564606936;, score=0.679 total time=   0.0s
[CV 2/5] END ...model__alpha=0.5016325564606936;, score=0.705 total time=   0.0s
[CV 3/5] END ...model__alpha=0.5016325564606936;, score=0.696 total time=   0.0s
[CV 4/5] END ...model__alpha=0.5016325564606936;, score=0.709 total time=   0.0s
[CV 5/5] END ...model__alpha=0.5016325564606936;, score=0.678 total time=   0.0s
[CV 1/5] END ...model__alpha=0.4229343632501507;, score=0.681 total time=   0.0s
[CV 2/5] END ...model__alpha=0.4229343632501507;

KeyboardInterrupt: 

Prétraitement: Retirer stopwords, sous-échantillonage aléatoire

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_10_iter", "stopwords_undersampling")

# Apprentissage par ensembles: Complement Naive Bayes, XGBoost, SVC, SGD

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'SVC', 'SGD']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_svc_sgd", "random_search_10_iter", "stopwords_undersampling_cumulative_sum")